# get_data

> async wrapper for asyncio requests


In [ ]:
# | default_exp client.get_data

In [ ]:
# | exporti
from typing import Callable, Optional, Union, Tuple, Any
from functools import wraps
import time

import httpx
import aiohttp
import asyncio


from pprint import pprint

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoError as de
import domolibrary.client.Logger as dl

In [ ]:
# | export


async def create_aiohttp_session(
    session: aiohttp.ClientSession = None,
) -> Tuple[aiohttp.ClientSession, bool]:
    is_close_session = False
    if session is None:
        is_close_session = True
        session = aiohttp.ClientSession()
    return session, is_close_session


def create_headers(
    auth: dmda.DomoAuth,  # The authentication object containing the Domo API token.
    content_type: dict = None,  # The content type for the request. Defaults to None.
    headers: dict = None,  # Any additional headers for the request. Defaults to None.
) -> dict:  # The headers for the request.
    """
    Creates default headers for interacting with Domo APIs.
    """
    if headers is None:
        headers = {}
    headers = {
        "Content-Type": content_type or "application/json",
        "Connection": "keep-alive",
        "accept": "application/json, text/plain",
        **headers,
    }
    if auth:
        headers.update(**auth.auth_header)
    return headers

In [ ]:
# | export
async def get_data_aiohttp(
    url: str,
    method: str,
    auth: dmda.DomoAuth,
    content_type: Optional[dict] = None,
    headers: Optional[dict] = None,
    # if no session passed by default will create and close session during execution
    session: Optional[aiohttp.ClientSession] = None,
    body: Union[dict, str, None] = None,
    params: Optional[dict] = None,
    debug_api: bool = False,
    process_stream: bool = False,
    stream_chunks: int = 10,
) -> rgd.ResponseGetData:
    """async wrapper for asyncio requests"""

    if auth and not auth.token:
        await auth.get_auth_token()

    session, is_close_session = await create_aiohttp_session(session)
    headers = create_headers(auth, content_type, headers)

    if debug_api:
        pprint(
            {
                "method": method,
                "url": url,
                "headers": headers,
                "json": body,
                "params": params,
            }
        )

    try:
        if headers.get("Content-Type") == "application/json":
            res = await session.request(
                method=method.upper(),
                url=url,
                headers=headers,
                json=body,
                params=params,
            )

        elif body is not None:
            res = await session.request(
                method=method.upper(),
                url=url,
                headers=headers,
                data=body,
                params=params,
            )

        else:
            res = await session.request(
                method=method.upper(), url=url, headers=headers, params=params
            )
        return await rgd.ResponseGetData._from_aiohttp_response(
            res, auth=auth, process_stream=process_stream, stream_chunks=stream_chunks
        )

    except Exception as e:
        print(e)

    finally:
        if is_close_session:
            await session.close()

#### sample implementation of get_data

During execution `get_data()` will attempt to retrieve exchange credentials for an auth token using the `dmda.DomoFullAuth.get_auth_token()` method.

Then the appropriate headers will be passed to the request.


In [ ]:
import os

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_username=os.environ["DOMO_USERNAME"],
    domo_password=os.environ["DOJO_PASSWORD"],
)

await full_auth.get_auth_token()

url = "https://domo-community.domo.com/api/content/v2/users/me"

try:
    res = await get_data_aiohttp(url=url, method="get", auth=full_auth)

except Exception as e:
    print(e)

assert res.is_success

In [ ]:
# |export
class GetData_Error(de.DomoError):
    def __init__(self, message, url):
        super().__init__(message=message, domo_instance=url)

In [ ]:
# | exporti
def create_httpx_session(
    session: httpx.AsyncClient = None,
) -> Tuple[httpx.AsyncClient, bool]:
    is_close_session = False
    if session is None:
        is_close_session = True
        session = httpx.AsyncClient()
    return session, is_close_session


async def handle_error(e, url, attempt, max_attempt):
    print(f"ℹ️ get_data error - {e} at {url}")
    attempt += 1
    if attempt == max_attempt:
        raise GetData_Error(url=url, message=e) from e
    await asyncio.sleep(5)
    return attempt

In [ ]:
# | export
async def get_data(
    url: str,
    method: str,
    auth: dmda.DomoAuth,
    content_type: Optional[dict] = None,
    headers: Optional[dict] = None,
    body: Union[dict, str, None] = None,
    params: Optional[dict] = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
    is_follow_redirects: bool = False,
    timeout=10,
    parent_class: str = None,  # name of the parent calling class
    num_stacks_to_drop: int = 2,  # number of stacks to drop from the stack trace.  see `domolibrary.client.Logger.TracebackDetails`.  use 2 with class > route structure.  use 1 with route based approach
    debug_traceback: bool = False,
) -> rgd.ResponseGetData:
    """async wrapper for asyncio requests"""

    if debug_api:
        print("🐛 debugging get_data")

    if auth and not auth.token:
        await auth.get_auth_token()

    headers = create_headers(auth=auth, content_type=content_type, headers=headers)

    session, is_close_session = create_httpx_session(session)

    traceback_details = dl.get_traceback(
        num_stacks_to_drop=num_stacks_to_drop,
        root_module="<module>",
        parent_class=parent_class,
        debug_traceback=debug_traceback,
    )

    if debug_api:
        pprint(
            {
                "parent_class": parent_class,
                "function_name": traceback_details.function_name,
                "method": method,
                "url": url,
                "headers": headers,
                "body": body,
                "params": params,
            }
        )

    attempt = 1
    max_attempt = 4

    while attempt <= max_attempt:
        try:
            if isinstance(body, dict) or isinstance(body, list):
                if debug_api:
                    print("get_data: sending json")
                res = await getattr(session, method.lower())(
                    url=url,
                    headers=headers,
                    json=body,
                    params=params,
                    follow_redirects=is_follow_redirects,
                    timeout=timeout,
                )

            elif body:
                if debug_api:
                    print("get_data: sending data")

                res = await getattr(session, method.lower())(
                    url=url,
                    headers=headers,
                    data=body,
                    params=params,
                    follow_redirects=is_follow_redirects,
                    timeout=timeout,
                )

            else:
                if debug_api:
                    print("get_data: no body")

                res = await getattr(session, method.lower())(
                    url=url,
                    headers=headers,
                    params=params,
                    follow_redirects=is_follow_redirects,
                    timeout=timeout,
                )

            if debug_api:
                print("get_data_response", res)

            if return_raw:
                return res

            return rgd.ResponseGetData._from_httpx_response(
                res, auth=auth, traceback_details=traceback_details
            )

        except (httpx.TransportError, RunTimeError) as e:
            attempt = await handle_error(e, url, attempt, max_attempt)
            session = httpx.AsyncClient()

        finally:
            if is_close_session:
                await session.aclose()

#### sample get_data

Notice that `get_data` returns a stack trace. with this route based approach pass 1 as `num_stacks_to_drop`


In [ ]:
import os
import pandas as pd

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


async def test_route_fn(auth: dmda.DomoAuth):
    url = f"https://{ auth.domo_instance}.domo.com/api/content/v2/users"

    res = await get_data(
        url=url,
        method="GET",
        auth=auth,
        debug_api=False,
        debug_traceback=False,
        num_stacks_to_drop=1,
    )
    return res


res = await test_route_fn(auth=auth)

from pprint import pprint

pprint({"traceback_details": res.traceback_details.__dict__})

pd.DataFrame(res.response[0:5])

{'traceback_details': {'file_name': '/tmp/ipykernel_12168/2519732825.py',
                       'function_name': 'test_route_fn',
                       'function_trail': '<module> -> test_route_fn',
                       'parent_class': None,
                       'traceback_stack': [<FrameSummary file /tmp/ipykernel_12168/2519732825.py, line 24 in <module>>,
                                           <FrameSummary file /tmp/ipykernel_12168/2519732825.py, line 13 in test_route_fn>]}}


,id,invitorUserId,displayName,userName,emailAddress,accepted,userType,timeZone,modified,created,role,roleId,rights,active,pending,systemUser,anonymous,title,department
0,0,2,monitor,monitor,monitor@domo.com,False,DOMO_SUPPORT,Etc/GMT+7,1588877394091,1464820854,Privileged,1,31.0,True,True,True,True,NaN,NaN
1,1003855998,583215149,Christine Hsieh,christine.hsieh@hellofresh.com,christine.hsieh@hellofresh.com,False,USER,NaN,1690383363000,1690382284,Privileged,2,31.0,True,True,False,True,NaN,NaN
2,1005321923,583215149,Matthew Lambourne,matthew.lambourne@frankandoak.com,matthew.lambourne@frankandoak.com,True,USER,NaN,1699463162302,1698668818,Privileged,2,31.0,True,False,False,False,Financial Controller,NaN
3,1006847540,1893952720,Marc-Anton Clavel,marcanton.clavel@domo.com,marcanton.clavel@domo.com,False,USER,NaN,1682078256937,1618579073,Privileged,2,31.0,True,True,False,True,Executive Analytics,Domo Client Services
4,1012895591,1893952720,JeMiller,JeMiller@marketaxess.com,JeMiller@marketaxess.com,True,USER,NaN,1657051684429,1657049419,NaN,2097317660,NaN,True,False,False,False,NaN,NaN


In [ ]:
import os
import pandas as pd

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


class Foo:
    async def test_route_fn(self, auth: dmda.DomoAuth):
        url = f"https://{ auth.domo_instance}.domo.com/api/content/v2/users"

        res = await get_data(
            url=url,
            method="GET",
            auth=auth,
            debug_api=False,
            debug_traceback=False,
            num_stacks_to_drop=1,
            parent_class=self.__class__.__name__,
        )
        return res


test_foo = Foo()
res = await test_foo.test_route_fn(auth=auth)

from pprint import pprint

pprint({"traceback_details": res.traceback_details.__dict__})

{'traceback_details': {'file_name': '/tmp/ipykernel_12168/2775044008.py',
                       'function_name': 'test_route_fn',
                       'function_trail': '<module> -> test_route_fn',
                       'parent_class': 'Foo',
                       'traceback_stack': [<FrameSummary file /tmp/ipykernel_12168/2775044008.py, line 27 in <module>>,
                                           <FrameSummary file /tmp/ipykernel_12168/2775044008.py, line 14 in test_route_fn>]}}


# get_data_stream


In [ ]:
# | export
async def get_data_stream(
    url: str,
    auth: dmda.DomoAuth,
    method: str = "GET",
    content_type: Optional[dict] = None,
    headers: Optional[dict] = None,
    # body: Union[dict, str, None] = None,
    params: Optional[dict] = None,
    debug_api: bool = False,
    timeout: int = 10,
    parent_class: str = None,  # name of the parent calling class
    num_stacks_to_drop: int = 2,  # number of stacks to drop from the stack trace.  see `domolibrary.client.Logger.TracebackDetails`.  use 2 with class > route structure.  use 1 with route based approach
    debug_traceback: bool = False,
) -> rgd.ResponseGetData:
    """async wrapper for asyncio requests"""

    if debug_api:
        print("🐛 debugging get_data")

    if auth and not auth.token:
        await auth.get_auth_token()

    if headers is None:
        headers = {}

    headers = {
        "Content-Type": content_type or "application/json",
        "Connection": "keep-alive",
        "accept": "application/json, text/plain",
        **headers,
    }

    if auth:
        headers.update(**auth.auth_header)

    traceback_details = dl.get_traceback(
        num_stacks_to_drop=num_stacks_to_drop,
        root_module="<module>",
        parent_class=parent_class,
        debug_traceback=debug_traceback,
    )

    if debug_api:
        pprint(
            {
                "method": method,
                "url": url,
                "headers": headers,
                # "body": body,
                "params": params,
                "traceback_details": traceback_details,
            }
        )

    attempt = 1
    max_attempt = 3

    content = bytearray()

    try:
        async with httpx.AsyncClient() as client:
            async with client.stream(method, url=url, headers=headers) as res:
                if res.status_code == 200:
                    async for chunk in res.aiter_bytes():
                        content += chunk

                return rgd.ResponseGetData(
                    status=res.status_code,
                    response=content,
                    is_success=True,
                    auth=auth,
                    traceback_details=traceback_details,
                )

    except httpx.TransportError as e:
        print(f"ℹ️ get_data error - {e} at {url}")
        attempt += 1

        if attempt == max_attempt:
            raise GetData_Error(url=url, message=e) from e

        await asyncio.sleep(5)

#### sample get_data_stream


In [ ]:
import os
import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

user_id = 1833256765
pixels = 300
debug_api = False

url = f"https://{token_auth.domo_instance}.domo.com/api/content/v1/avatar/USER/{user_id}?size={pixels}"

res = await get_data_stream(
    url=url,
    method="GET",
    auth=token_auth,
    debug_api=False,
    headers={"accept": "image/png;charset=utf-8"},
    num_stacks_to_drop=0,
)

folder_path = "../test"

if not os.path.exists(folder_path):
    os.mkdir(folder_path)

local_filename = f"{folder_path}/{user_id}.png"

with open(local_filename, "wb") as f:
    f.write(res.response)

res

ResponseGetData(status=200, response=bytearray(b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x01,\x08\x02\x00\x00\x00\xf6\x1f\x19"\x00\x00\x80\x00IDATx^\xbc\x9d\x85[\x1cI\xbb\xf6\xd9\x8d\x13\xc3\xddu\x18`\x98\xc1apwwwK \x10\xe2\xee.\xeb.\xd9lv\xb3q\x17\x12\x9c@\x08\x0e\x01\xe2\xc9\xca\xfb\x9es\xfe\x8c\xef\xe9\xae\xa1\xa8\xa9\xea\x19H\xce{\xbe\xeb\xba\xaf\xbe\x9e\xae\xae\xee\xe9I\xea\xc7\xfd<\xd52\x1a\x9d\x1dW@\xdd\x1dW{:\xae\xf6v\\{\xd4~\xb5\xb7\xfdj_\xc7\xb5\xfe\x8e\xeb\x8f\xbbn\xf4u^\xc7\xe2\xb6v\\\xeb\xe9\xa4\xf5\xa8\xeb:Hp\x15\x82\xde\xee\x1b\xa0\xbe\x9e\x9b\xaa\xf4\xe4\xd1\x8d\xfe\x9e\xeb\x8f\xbb\xaf\xc1\x92\x8b{o\x83\x1e?\xba\x05\x82\xad(@B\x9b\x9e\xf4\xdd\x01\xa1\x00\xf5\x84`\xe0\xf1\xdd\xc1\xbe[XC\x8fo\x0f\xf4\xde\x84%\xd2\xc8\x93\xbb\xc3\xfdw@\x10p\xf1\x93\xbbCO\xee\x83\x86\x07Z@#\x83\x0f@(\x18\x1dz\x88[ \x06\xa1`l\xb8UP\xe3#mXO\xc7:&F\xdb!\x80%\xe8\xe9\xc8C\xd0\xe4h\xeb\xd4X\x1b\x08\x02R\xd3\xe3\xed\xa4\xa0\x03\xd5\x02z6\xd1\x817A\xcc\xadNtLM\xf4L?\xed\xc6\x82\xc

# Looper


In [ ]:
# | export
class LooperError(Exception):
    def __init__(self, loop_stage: str, message):
        super().__init__(f"{loop_stage} - {message}")

In [ ]:
# | export
async def looper(
    auth: dmda.DomoAuth,
    session: httpx.AsyncClient,
    url,
    offset_params,
    arr_fn: callable,
    loop_until_end: bool = False,  # usually you'll set this to true.  it will override maximum
    method="POST",
    body: dict = None,
    fixed_params: dict = None,
    offset_params_in_body: bool = False,
    body_fn=None,
    limit=1000,
    skip=0,
    maximum=0,
    debug_api: bool = False,
    debug_loop: bool = False,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
    timeout: bool = 10,
    wait_sleep: int = 0,
) -> rgd.ResponseGetData:
    is_close_session = False

    session, is_close_session = create_httpx_session(session)

    allRows = []
    isLoop = True

    res = None

    if maximum and maximum <= limit and not loop_until_end:
        limit = maximum

    while isLoop:
        params = fixed_params or {}

        if offset_params_in_body:
            body.update(
                {offset_params.get("offset"): skip, offset_params.get("limit"): limit}
            )

        else:
            params.update(
                {offset_params.get("offset"): skip, offset_params.get("limit"): limit}
            )

        if body_fn:
            try:
                body = body_fn(skip, limit, body)

            except Exception as e:
                await session.aclose()
                raise LooperError(
                    loop_stage="processing body_fn", message=str(e)
                ) from e

        if debug_loop:
            print(f"\n🚀 Retrieving records {skip} through {skip + limit} via {url}")
            # pprint(params)

        res = await get_data(
            auth=auth,
            url=url,
            method=method,
            params=params,
            session=session,
            body=body,
            debug_api=debug_api,
            timeout=timeout,
            parent_class=parent_class,
            num_stacks_to_drop=debug_num_stacks_to_drop,
        )

        if not res.is_success:
            if is_close_session:
                await session.aclose()

            return res

        try:
            newRecords = arr_fn(res)

        except Exception as e:
            await session.aclose()
            raise LooperError(loop_stage="processing arr_fn", message=str(e)) from e

        allRows += newRecords

        if len(newRecords) == 0:
            isLoop = False

        if maximum and len(allRows) >= maximum and not loop_until_end:
            isLoop = False

        if debug_loop:
            print({"all_rows": len(allRows), "new_records": len(newRecords)})
            print(f"skip: {skip}, limit: {limit}")

        if maximum and skip + limit > maximum and not loop_until_end:
            limit = maximum - len(allRows)

        skip += len(newRecords)
        time.sleep(wait_sleep)

    if debug_loop:
        print(
            f"\n🎉 Success - {len(allRows)} records retrieved from {url} in query looper\n"
        )

    if is_close_session:
        await session.aclose()

    return await rgd.ResponseGetData._from_looper(res=res, array=allRows)

#### sample implementation of looper


In [ ]:
# | hide
import os
import pandas as pd

session = httpx.AsyncClient()

sql = "SELECT * FROM TABLE"
dataset_id = os.environ["DOJO_DATASET_ID"]

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

url = f"https://{token_auth.domo_instance}.domo.com/api/query/v1/execute/{dataset_id}"

offset_params = {
    "offset": "offset",
    "limit": "limit",
}


def body_fn(skip, limit, body=None):
    return {"sql": f"{sql} limit {limit} offset {skip}"}


def arr_fn(res: rgd.ResponseGetData):
    rows_ls = res.response.get("rows")
    columns_ls = res.response.get("columns")
    output = []
    for row in rows_ls:
        new_row = {}
        for index, column in enumerate(columns_ls):
            new_row[column] = row[index]
        output.append(new_row)
    return output


res = await looper(
    auth=token_auth,
    method="POST",
    url=url,
    offset_params=offset_params,
    skip=0,
    maximum=99,
    limit=100,
    arr_fn=arr_fn,
    body_fn=body_fn,
    debug_api=False,
    debug_loop=True,
    loop_until_end=False,
    session=session,
)

await session.aclose()

print(len(res.response))
pd.DataFrame(res.response[0:5])


🚀 Retrieving records 0 through 99 via https://domo-community.domo.com/api/query/v1/execute/04c1574e-c8be-4721-9846-c6ffa491144b
{'all_rows': 99, 'new_records': 99}
skip: 0, limit: 99

🎉 Success - 99 records retrieved from https://domo-community.domo.com/api/query/v1/execute/04c1574e-c8be-4721-9846-c6ffa491144b in query looper

99


,objectID,url,Title,article,views,created_dt,published_dt
0,000004785,https://domo-support.domo.com/s/article/360047...,Backing Up Workbench 4 Jobs,Important: Support for Workbench 4 ended on ...,138,2022-10-24T22:30:00,2022-10-24T22:42:00
1,000004807,https://domo-support.domo.com/s/article/360044...,Backing Up Workbench 5 Jobs,Backing up DataSet jobs is an often overlooked...,47,2022-10-24T22:31:00,2022-10-24T22:41:00
2,000004785,https://domo-support.domo.com/s/article/360047...,Backing Up Workbench 4 Jobs,Important: Support for Workbench 4 ended on ...,139,2022-10-24T22:30:00,2022-10-24T22:42:00
3,000004081,https://domo-support.domo.com/s/article/360043...,Beast Mode Functions Reference Guide,IntroYou can use this reference guide to learn...,826,2022-10-24T21:20:00,2022-10-24T22:40:00
4,000004508,https://domo-support.domo.com/s/article/360043...,Fun Sample DataSets,IntroIt's hard learning how to perform advance...,365,2022-10-24T22:13:00,2022-10-24T22:39:00


## Aiohttp Looper DEPRECATED


In [ ]:
# | export
async def looper_aiohttp(
    auth: dmda.DomoAuth,
    session: aiohttp.ClientSession,
    url,
    offset_params,
    arr_fn: callable,
    loop_until_end: bool = False,
    method="POST",
    body: dict = None,
    fixed_params: dict = None,
    offset_params_in_body: bool = False,
    body_fn=None,
    limit=1000,
    skip=0,
    maximum=2000,
    debug_api: bool = False,
    debug_loop: bool = False,
) -> rgd.ResponseGetData:
    is_close_session = False

    if not session:
        session = aiohttp.ClientSession()
        is_close_session = True

    allRows = []
    isLoop = True

    res = None

    if maximum < limit:
        limit = maximum

    while isLoop:
        params = fixed_params or {}

        if offset_params_in_body:
            body[offset_params.get("offset")] = skip
            body[offset_params.get("limit")] = limit

        else:
            params[offset_params.get("offset")] = skip
            params[offset_params.get("limit")] = limit

        if body_fn:
            try:
                body = body_fn(skip, limit)

            except Exception as e:
                await session.aclose()
                raise LooperError(
                    loop_stage="processing body_fn", message=str(e)
                ) from e

        if debug_loop:
            print(f"\n🚀 Retrieving records {skip} through {skip + limit} via {url}")
            # pprint(params)

        res = await get_data_aiohttp(
            auth=auth,
            url=url,
            method=method,
            params=params,
            session=session,
            body=body,
            debug_api=debug_api,
        )

        if not res.is_success:
            if is_close_session:
                await session.aclose()
            return res

        try:
            newRecords = arr_fn(res)

        except Exception as e:
            await session.close()
            raise LooperError(loop_stage="processing arr_fn", message=str(e)) from e

        allRows += newRecords

        if loop_until_end and len(newRecords) != 0:
            maximum = maximum + limit

        if debug_loop:
            print({"all_rows": len(allRows), "new_records": len(newRecords)})

        if len(allRows) >= maximum or len(newRecords) == 0:
            if debug_loop:
                print(
                    f"\n🎉 Success - {len(allRows)} records retrieved from {url} in query looper\n"
                )

            break

        skip += len(newRecords)

        if skip + limit > maximum:
            limit = maximum - len(allRows)

            if debug_loop:
                print(f"skip: {skip}, limit: {limit}")

    if is_close_session:
        await session.close()

    return await rgd.ResponseGetData._from_looper(res=res, array=allRows)

In [ ]:
# | hide
import os
import pandas as pd

session = aiohttp.ClientSession()

sql = "SELECT * FROM TABLE"
dataset_id = os.environ["DOJO_DATASET_ID"]

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

url = f"https://{token_auth.domo_instance}.domo.com/api/query/v1/execute/{dataset_id}"

offset_params = {
    "offset": "offset",
    "limit": "limit",
}

maximum = 10
skip = 0


def body_fn(skip, limit):
    return {"sql": f"{sql} limit {limit} offset {skip}"}


def arr_fn(res: rgd.ResponseGetData):
    rows_ls = res.response.get("rows")
    columns_ls = res.response.get("columns")
    output = []
    for row in rows_ls:
        new_row = {}
        for index, column in enumerate(columns_ls):
            new_row[column] = row[index]
        output.append(new_row)
    return output


res = await looper_aiohttp(
    auth=token_auth,
    method="POST",
    url=url,
    offset_params=offset_params,
    skip=skip,
    maximum=maximum,
    arr_fn=arr_fn,
    body_fn=body_fn,
    debug_api=False,
    debug_loop=False,
    loop_until_end=False,
    session=session,
)

await session.close()

pd.DataFrame(res.response[0:5])

,objectID,url,Title,article,views,created_dt,published_dt
0,000004785,https://domo-support.domo.com/s/article/360047...,Backing Up Workbench 4 Jobs,Important: Support for Workbench 4 ended on ...,138,2022-10-24T22:30:00,2022-10-24T22:42:00
1,000004807,https://domo-support.domo.com/s/article/360044...,Backing Up Workbench 5 Jobs,Backing up DataSet jobs is an often overlooked...,47,2022-10-24T22:31:00,2022-10-24T22:41:00
2,000004785,https://domo-support.domo.com/s/article/360047...,Backing Up Workbench 4 Jobs,Important: Support for Workbench 4 ended on ...,139,2022-10-24T22:30:00,2022-10-24T22:42:00
3,000004081,https://domo-support.domo.com/s/article/360043...,Beast Mode Functions Reference Guide,IntroYou can use this reference guide to learn...,826,2022-10-24T21:20:00,2022-10-24T22:40:00
4,000004508,https://domo-support.domo.com/s/article/360043...,Fun Sample DataSets,IntroIt's hard learning how to perform advance...,365,2022-10-24T22:13:00,2022-10-24T22:39:00


In [ ]:
# | export


class RouteFunction_ResponseTypeError(TypeError):
    def __init__(self, result):
        super().__init__(
            f"Expected function to return an instance of ResponseGetData got {type(result)} instead.  Refactor function to return ResponseGetData class"
        )


def route_function(func: Callable[..., Any]) -> Callable[..., Any]:
    """
    Decorator for route functions to ensure they receive certain arguments.
    If these arguments are not provided, default values are used.

    Args:
        func (Callable[..., Any]): The function to decorate.

    Returns:
        Callable[..., Any]: The decorated function.

    The decorated function takes the following arguments:
        *args (Any): Positional arguments for the decorated function.
        parent_class (str, optional): The parent class. Defaults to None.
        debug_num_stacks_to_drop (int, optional): The number of stacks to drop for debugging. Defaults to 1.
        debug_api (bool, optional): Whether to debug the API. Defaults to False.
        session (httpx.AsyncClient, optional): The HTTPX client session. Defaults to None.
        **kwargs (Any): Additional keyword arguments for the decorated function.
    """

    @wraps(func)
    async def wrapper(
        *args: Any,
        parent_class: str = None,
        debug_num_stacks_to_drop: int = 1,
        debug_api: bool = False,
        session: httpx.AsyncClient = None,
        **kwargs: Any,
    ) -> Any:
        result = await func(
            *args,
            parent_class=parent_class,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
            debug_api=debug_api,
            session=session,
            **kwargs,
        )

        if not isinstance(result, rgd.ResponseGetData):
            raise RouteFunction_ResponseTypeError(result)

        return result

    return wrapper

#### sample test RouteFunction_ResponseTypeError 

In [ ]:
try:
    raise RouteFunction_ResponseTypeError("hello world")

except RouteFunction_ResponseTypeError as e:
    print(e)

Expected function to return an instance of ResponseGetData got <class 'str'> instead.  Refactor function to return ResponseGetData class


In [ ]:
@route_function
async def test_fn(parent_class = None, debug_num_stacks_to_drop = 1, debug_api = True, session = None):

    return rgd.ResponseGetData(status = 200, response = 100, is_success = True)

await test_fn()

ResponseGetData(status=200, response=100, is_success=True, parent_class=None)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
!nbqa black 10_get_data.ipynb

reformatted 10_get_data.ipynb

All done! ✨ 🍰 ✨
1 file reformatted.
